In [6]:
import pandas as pd
import sqlalchemy as db

# create sqlalchemy engine
engine = db.create_engine('postgresql+psycopg2://postgres:pw@localhost/eu')

#1 read from csv and write to database
'''
df = pd.read_csv('C:\\Users\\asuer\\Documents\\Google Drive\\ebs 2001-2005\\ebs63.1_2005.csv', engine='python')
df.columns = [c.lower() for c in df.columns] #postgres doesn't like capitals or spaces
#write to databas
df.to_sql("eu_2005", engine)

df2 = pd.read_csv('C:\\Users\\asuer\\Documents\\Google Drive\\ebs 2001-2005\\ebs73.1_2010.csv', engine='python')
df2.columns = [c.lower() for c in df2.columns] #postgres doesn't like capitals or spaces
df2.to_sql("eu_2010", engine)

df3 = pd.read_csv('C:\\Users\\asuer\\Documents\\Google Drive\\ebs 2001-2005\\ebs55.2_2001.csv', engine='python')
df3.columns = [c.lower() for c in df3.columns] #postgres doesn't like capitals or spaces
df3.to_sql("eu_2001", engine)

df4 = pd.read_csv('C:\\Users\\asuer\\Documents\\Google Drive\\ebs 2001-2005\\ebscandidates2002.csv', engine='python')
df4.columns = [c.lower() for c in df4.columns] #postgres doesn't like capitals or spaces
df4.to_sql("eu_2002", engine)

'''

#2. retreive the tables
connection = engine.connect()
metadata = db.MetaData()
eu_2001 = db.Table('eu_2001', metadata, autoload=True, autoload_with=engine)
eu_2002 = db.Table('eu_2002', metadata, autoload=True, autoload_with=engine)
eu_2005 = db.Table('eu_2005', metadata, autoload=True, autoload_with=engine)
eu_2010 = db.Table('eu_2010', metadata, autoload=True, autoload_with=engine)


In [8]:
''' 

3. make queries and select required files needs flask (or dash) back end to run the codes, and a front end 
returns a pandas DF with the required files: 
GUI:
files to retrieve? dropdown1: names of the files in the database, when one is selected dropdown2: 
names of the variables in the file 
3.1. when a vraible is selected, ask if the user wnats to rename it. for example v7=country 
3.2. asks if a new variable w a constant value to be entered for example year=2002
3.3 asks if there are any specific values to filter out (range or discrete values)
codes for each step below
3.4 when the df is finalized, ask the user if new variables to be created. for example: continent from country

'''
#3.0
query=db.select([eu_2001.columns.isocntry, eu_2001.columns.v117, eu_2001.columns.v118, eu_2001.columns.v119, eu_2001.columns.v120, eu_2001.columns.v121, eu_2001.columns.v122])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)

query=db.select([eu_2002.columns.isocntry, eu_2002.columns.q14_1, eu_2002.columns.q14_2, eu_2002.columns.q14_3, 
                 eu_2002.columns.q14_4, eu_2002.columns.q14_5, eu_2002.columns.q14_6])

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df2 = pd.DataFrame(ResultSet)

query=db.select([eu_2005.columns.v7, eu_2005.columns.v231, eu_2005.columns.v232, eu_2005.columns.v233, 
                 eu_2005.columns.v234, eu_2005.columns.v235, eu_2005.columns.v236])

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df3 = pd.DataFrame(ResultSet)


query=db.select([eu_2010.columns.v7, eu_2010.columns.v322,  eu_2010.columns.v324, 
                 eu_2010.columns.v325, eu_2010.columns.v326,eu_2010.columns.v327, eu_2010.columns.v328 ])

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df4 = pd.DataFrame(ResultSet)

#3.1
df.rename(columns={0: 'country',1: 'health',2: 'resources',  3: 'sort', 4: 'faith', 5: 'env',6: 'animals' }, inplace=True)
df['year'] = '2002'#3.2
df=df.infer_objects()
df.where((df.loc[:, 'health':'animals'] <3), inplace=True)#3.3

df2.rename(columns={0: 'country',1: 'health',2: 'resources',  3: 'sort', 4: 'faith', 5: 'env',6: 'animals' }, inplace=True)
df2['year'] = '2002'
df2=df2.infer_objects()
df2.where((df2.loc[:, 'health':'animals'] <3), inplace=True)

df3.rename(columns={0: 'country',1: 'health',2: 'resources',  3: 'sort', 4: 'faith', 5: 'env',6: 'animals' }, inplace=True)
df3['year'] = '2005'
df3=df3.infer_objects()
df3.where((df3.loc[:, 'health':'animals'] <6), inplace=True)


df4.rename(columns={0: 'country',1: 'health',2: 'resources',  3: 'sort', 4: 'faith', 5: 'env',6: 'animals'}, inplace=True)
df4['year'] = '2010'
df4=df4.infer_objects()
df4.where((df4.loc[:, 'health':'animals'] <6), inplace=True)


# Final dataframe

DF = df.append([df2, df3, df4])
dfm=pd.melt(DF, id_vars=['country', 'year'], value_vars=DF.loc[:,'health':'animals'])
dfin=dfm.groupby(['country', 'year','variable']).mean().reset_index()
dfin['year']=dfin['year'].astype(int)


dfin.rename(columns={'variable': 'Indicator Name', 'value': 'Value', 'year': 'Year', 'country':'Country Name'}, inplace=True)

#3.4
dfin['continent']=dfin['Country Name'].replace({'SK': 'East', 'CZ': 'East', 'SI':'East','RO' :'East', 'LV':'East','ES':'Med','FR':'West','AT':'West',
                    'BE':'West', 'PT':'Med', 'SE':'Scan', 'LT':'East', 'CH': 'West', 'IT':'Med', 'NL':'West', 
                    'DK': 'Scan','BG':'East', 'HU':'East', 'IE':'West','GB-GBN':'West', 'FI': 'Scan', 'TR':'Med', 
                    'NO': 'Scan', 'DE-W':'West', 'EE':'East', 'GR':'Med', 'HR':'East','PL':'East','DE-E':'East', 
                    'LU':'West', 'CY':'Med', 'IS': 'Scan', 'MT':'Med', 'GB-NIR':'West'})#3.4




In [10]:
#4. get the dash GUI, embed in the website and deploy at a server 
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


available_indicators = dfin['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='resources'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='sort'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=dfin['Year'].min(),
        max=dfin['Year'].max(),
        value=dfin['Year'].max(),
        marks={str(year): str(year) for year in dfin['Year'].unique()},
        step=None
    )
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value'),
     Input('year--slider', 'value')])


def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type, 
                 year_value):
    dff = dfin[dfin['Year'] == year_value]
    traces = []
    for i in dff.continent.unique():
        df_by_continent = dff[dff['continent'] == i]
        traces.append(dict(
            x=df_by_continent[df_by_continent['Indicator Name'] == xaxis_column_name]['Value'],
            y=df_by_continent[df_by_continent['Indicator Name'] == yaxis_column_name]['Value'],
            text=df_by_continent[df_by_continent['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            },
            name=i           
        ))

    return {
        'data': traces,
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }



app.run_server(debug=True, use_reloader=False)


Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 110-447-508
Debugger PIN: 110-447-508
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
